In [1]:
import datetime
import intake
import warnings 
import hvplot.pandas
import hvplot.xarray
import cartopy.crs as ccrs
from holoviews.operation.datashader import rasterize
import geoviews as gv

import warnings
warnings.filterwarnings('ignore')

**Open the NWM intake catalog**

In [2]:

catalog = intake.open_catalog('s3://noaa-nodd-kerchunk-pds/nwm_ll/nwm_intake.yml',storage_options={'anon':True})
(list(catalog))

['NWM_Best_CONUS_Short_Range',
 'NWM_Best_CONUS_Medium_Range',
 'NWM_Best_Alaska_Short_Range',
 'NWM_Best_Alaska_Medium_Range',
 'NWM_Best_Hawaii_Short_Range',
 'NWM_Best_PuertoRico_Short_Range']


&nbsp;

**Select the catalog entry to open**

&nbsp;
Open one of the catalog entries and get back a Xarray dataset

In [3]:
cat = catalog['NWM_Best_CONUS_Short_Range']
ds = cat.to_dask()
ds

<xarray.Dataset> Size: 97GB
Dimensions:         (feature_id: 2776734, time: 726, reference_time: 1)
Coordinates:
  * feature_id      (feature_id) int64 22MB 101 179 ... 1180001803 1180001804
    latitude        (feature_id) float32 11MB dask.array<chunksize=(2776734,), meta=np.ndarray>
    longitude       (feature_id) float32 11MB dask.array<chunksize=(2776734,), meta=np.ndarray>
  * reference_time  (reference_time) datetime64[ns] 8B 2024-08-06T23:00:00
  * time            (time) datetime64[ns] 6kB 2024-08-07 ... 2024-09-06T05:00:00
Data variables:
    altitude        (feature_id) float32 11MB dask.array<chunksize=(2776734,), meta=np.ndarray>
    crs             (time) object 6kB dask.array<chunksize=(1,), meta=np.ndarray>
    from_link_id    (feature_id) float64 22MB dask.array<chunksize=(2776734,), meta=np.ndarray>
    link_id         (feature_id) float64 22MB dask.array<chunksize=(2776734,), meta=np.ndarray>
    nudge           (time, feature_id) float64 16GB dask.array<chunksize=(1, 2776734), meta=np.ndarray>
    qBtmVertRunoff  (time, feature_id) float64 16GB dask.array<chunksize=(1, 2776734), meta=np.ndarray>
    qBucket         (time, feature_id) float64 16GB dask.array<chunksize=(1, 2776734), meta=np.ndarray>
    qSfcLatRunoff   (time, feature_id) float64 16GB dask.array<chunksize=(1, 2776734), meta=np.ndarray>
    streamflow      (time, feature_id) float64 16GB dask.array<chunksize=(1, 2776734), meta=np.ndarray>
    terminal_node   (feature_id) float64 22MB dask.array<chunksize=(2776734,), meta=np.ndarray>
    to_link_id      (feature_id) float64 22MB dask.array<chunksize=(2776734,), meta=np.ndarray>
    velocity        (time, feature_id) float64 16GB dask.array<chunksize=(1, 2776734), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.6
    NWM_version_number:         v3.0
    TITLE:                      OUTPUT FROM NWM v3.0
    cdm_datatype:               Station
    code_version:               v5.3.0-alpha1
    dev:                        dev_ prefix indicates development/internal me...
    ...                         ...
    model_output_type:          channel_rt
    model_output_valid_time:    2024-08-07_00:00:00
    model_total_valid_times:    18
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    station_dimension:          feature_id
    stream_order_output:        1




&nbsp;
**Set the spatial bounds and time to extract**

In [4]:
lon_min =-100
lon_max=-60

lat_min=20
lat_max=60
time_s=datetime.datetime.now()


&nbsp;

**Extract the data**

&nbsp;
Compute what feature_ids are within the bounding box

&nbsp;
Extract the selected time and the computed feature ids from the dataset



In [5]:
%%time
ids = ((ds.latitude > lat_min) & (ds.latitude < lat_max) & (ds.longitude > lon_min) & (ds.longitude < lon_max)).compute()
s_ds = ds.sel(time=time_s, method='nearest').isel(feature_id=ids)

stream_ds = s_ds['streamflow'].compute()

stream_ds

CPU times: user 1.32 s, sys: 292 ms, total: 1.61 s
Wall time: 6.08 s


<xarray.DataArray 'streamflow' (feature_id: 1689578)> Size: 14MB
array([0.54999999, 0.        , 0.        , ..., 0.01      , 0.        ,
       0.        ])
Coordinates:
  * feature_id  (feature_id) int64 14MB 101 179 181 ... 1131002853 1131002854
    latitude    (feature_id) float32 7MB 31.09 46.02 46.02 ... 25.78 25.76 25.76
    longitude   (feature_id) float32 7MB -94.64 -67.99 -68.0 ... -97.67 -97.68
    time        datetime64[ns] 8B 2024-09-05T09:00:00
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1
    valid_range:   [0, 5000000]

&nbsp;

**Plot the extracted data**



Convert xarray to pandas dataframe

In [6]:


df=stream_ds.to_pandas().to_frame()
df = df.assign(longitude = stream_ds.longitude)
df =df.assign(latitude=stream_ds.latitude)

df.rename(columns={0:"streamflow"},inplace=True)


Rasterize and plot the data with hvplot

In [7]:

p = df.hvplot.points('longitude', 'latitude', crs=ccrs.PlateCarree(),
                     c='streamflow',size=14)
g = rasterize(p, aggregator='mean', x_sampling=0.001, y_sampling=0.001).opts(tools=['hover'], 
                 aspect='equal',colorbar=True,logz=True, cmap='viridis', clim=(1e-2, 2000), width=500,height=400 )

g  * gv.tile_sources.OSM

:DynamicMap   []
   :Overlay
      .Image.I :Image   [longitude,latitude]   (longitude_latitude streamflow)
      .WMTS.I  :WMTS   [Longitude,Latitude]